# Autodiff Tutorial

In this tutorial we'll be implementing a minimal autodiff library.

## Credits, reading/watching material


It is heavily based on the [autodiff](https://mdrk.io/introduction-to-automatic-differentiation/) [tutorials](https://mdrk.io/introduction-to-automatic-differentiation-part2/) by [Marcus D. R. Klarqvist](https://mdrk.io/). I strongly encourage you to carefully read those tutorials, which include deeper explanations.

Other resources I studied to make this tutorial
* [micrograd](https://github.com/karpathy/micrograd) by [Andrej Karpathy](https://karpathy.ai/)
* https://douglasorr.github.io/2021-11-autodiff/article.html
* https://sidsite.com/posts/autodiff/
* [video series](https://www.youtube.com/watch?v=RxmBukb-Om4&list=PLeDtc0GP5ICldMkRg-DkhpFX1rRBNHTCs) by [Joel Grus](https://joelgrus.com/)

# Differentiation rules (a refresher)


Let's remember some of the rules of differentiation:

## Linear rules ([differentiation is linear](https://en.wikipedia.org/wiki/Linearity_of_differentiation))

For any functions $f$ and $g$ and any real numbers $a$ and $b$, the derivative of the function $h(x) = af(x) + bg(x)$ w.r.t $x$ is:

$h'(x) = af'(x) + bg'|(x)$

some special cases are:

* constant factor $h(x) = af(x)$
  
  $h'(x) = af'(x)$

* Sum $h(x) = f(x) + g(x)$

  $h'(x) = f'(x) + g'(x)$

* Subtraction $h(x) = f(x) - g(x)$

  $h'(x) = f'(x) - g'(x)$

### Product rule

If $h(x) = f(x)g(x)$:

$h'(x) = f'(x)g(x) + f(x)g'(x)$

### Quotient rule

If $\displaystyle h(x) = \frac{f(x)}{g(x)}$:

$\displaystyle h'(x) = \frac{f'(x)g(x) - f(x)g'(x)}{g(x)^2}$


### Chain rule

The main player of the autodifferentiation idea! If $h(x) = f(g(x))$:

$h'(x) = f'(g(x)) \cdot f(x)g'(x)$

A common variable replacement and using [Leibniz's notation](https://en.wikipedia.org/wiki/Leibniz%27s_notation). if $y = f(u)$ and $u = g(x)$:

$\displaystyle\frac{\mathrm{d}y}{\mathrm{d}x} = \frac{\mathrm{d}y}{\mathrm{d}u}\cdot\frac{\mathrm{d}u}{\mathrm{d}x}$

# Implementing Automatic differentiation

Using the chain rule, we can build a system that automatically converts a function into a sequence of operations for which we know the derivative.

Modern autodiff can be implemented in two ways:
* **forward-mode:** The derivatives are calculated in a single pass through the computation graph.
* **reverse-mode:** The derivatives are calculated starting from the output, and traversing the computation graph in reverse order.

> Throughout this tutorial, we will be implementing a `Variable` class using [operator overloading](https://en.wikipedia.org/wiki/Operator_overloading), which in python requires implementing [special names](https://docs.python.org/3/reference/datamodel.html#special-method-names) for some of the basic arithmetic operators. 

In general, once you fully grasp the idea of function composition and the chain rule, you will realize that we can effectively implement a differentiation engine with basically 2 lines of code per operation we want to support!

# Forward-mode autodiff

While the end goal is to implement reverse-mode, we must understand the forward-mode first. Maybe the best way to understand it is to implement it!

In [ ]:
import numpy as np

In [ ]:
class Variable():
    def __init__(self, f, d = 0):
        self.f = f # the function evaluation: f(x)
        self.d = d # the derivative: f'(x)
    
    def __repr__(self):
        # this function allows us to see the values of the class members when printing
        return f"Var(f={self.f}, d={self.d})"

let's test the creation of a new variable

In [ ]:
a = Variable(1, 2)
a

Let's overload our first arithmetic operator. Where possible, I will use [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions), which allows us to define anonymous functions with less code.

In [ ]:
# sum
Variable.__add__ = lambda self, other: Variable(self.f + other.f, self.d + other.d)

We can test this function

In [ ]:
a = Variable(1, 2)
b = Variable(3, 4)
print(f"    a = {a}")
print(f"    b = {b}")
print(f"a + b = {a + b}")
print(f"b + a = {b + a}")

And now we can add (and test), the remaining basic operators

In [ ]:
# subtraction
Variable.__sub__ = lambda self, other: Variable(self.f - other.f, self.d - other.d)

# multiplication 
# notice that here the derivative already used both f and d
Variable.__mul__ = lambda self, other: Variable(self.f * other.f, self.d * other.f + self.f * other.d)

# division
Variable.__truediv__ = lambda self, other: Variable(self.f / other.f, (self.d * other.f - self.f * other.d) / other.f ** 2)

Now let's test if our little implementation composes into a sensible derivative for a simple function:

$$f(x, a, b) = \frac{xa}{(x^2 + b)}$$

Using the differentiation rules from above, we can (analitically) determine that the derivative of $f$ with respect to $x$ is:

$$\frac{\partial f(x, a, b)}{\partial x} = \frac{a}{x^2 + b} - \frac{2ax^2}{(x^2 + b)^2}$$

If we set values for each of these variables, we can compute $f(x, a, b)$ and $\displaystyle\frac{\partial f(x, a, b)}{\partial x}$ numerically, to verify that the computation is correct. Let's set the values 

* $x = 24$
* $a = 22$
* $b = 2$


In [ ]:
x = 24
a = 22
b = 2

# f(x, a, b)
f = x * a / (x * x + b)
d_x = a/(x ** 2 + b) - (2 * a * x ** 2)/(x**2 + b)**2
print(f"f = {f}")
print(f"d_x = {d_x}")

and now let's see if our differentiation engine computes these values correctly

In [ ]:
x = Variable(24, 1)
a = Variable(22, 0)
b = Variable(2, 0)
x * a / (x * x + b)

Now let's add more operators to our engine:

In [ ]:
Variable.sin = lambda self: Variable(np.sin(self.f), np.cos(self.f) + self.d)
Variable.cos = lambda self: Variable(np.cos(self.f), -np.sin(self.f) + self.d)
Variable.tan = lambda self: self.sin()/self.cos() # notice that by implementing this identity, we already get the derivative for free.
Variable.log = lambda self: Variable(np.log(self.f), 1 / self.f * self.d)
Variable.exp = lambda self: Variable(np.exp(self.f), np.exp(self.f) * self.d)

def var_pow(self, number):
    assert isinstance(number, (int, float))
    return Variable(np.power(self.f, number), number * np.power(self.f, number -1) * self.d)

Variable.__pow__ = var_pow

These are unary operators (they operate on the same variable). Notice how we always multiply by `self.d`. The reason for this is that we are representing each variable as a [dual number](https://en.wikipedia.org/wiki/Dual_number). We don't go very deep into that in this tutorial, but it suffices to know that they are similar to complex numbers, but they have the form $a + b\varepsilon$, where $\varepsilon$ is a symbol that satisfies $\varepsilon^2 = 0$. We also need to know that extending the derivative to dual numbers, and we evaluate it at $x+1\varepsilon$, we satisfy:

$$f(x + 1\varepsilon) = f(x) + f'(x)\varepsilon$$

This is the theoretical reason why forward-mode autodiff works. By computing a function and evaluating it on $x+1\varepsilon$, we are automatically computing the derivative. 

Now, let's go back to the function we differentiated before:

$$f(x, a, b) = \frac{xa}{(x^2 + b)}$$

If we want to determine the derivative of $f$ with respect to $a$ analitically, we get:

$$\frac{\partial f(x, a, b)}{\partial a} = \frac{x}{x^2 + b}$$

and with respect to $b$:

$$\frac{\partial f(x, a, b)}{\partial b} = \frac{-xa}{(x^2 + b)^2}$$

Numerically:

In [ ]:
x = 24
a = 22
b = 2

# f(x, a, b)
f = x * a / (x * x + b)
d_a = x/(x ** 2 + b)
d_b = -x * a / (x ** 2 + b) ** 2

print(f"  f = {f}")
print(f"d_x = {d_x}") # we calculated this above
print(f"d_a = {d_a}")
print(f"d_b = {d_b}")


If we look again at how we defined the variables before, we see that in order to calculate the derivative with respect to a single variable, we need to set the gradient of each variable to 1 and the remaining ones to 0 each time, let's do that and compare:

In [ ]:
def f(x, a, b):
    return x * a / (x ** 2 + b) # because we implemented __pow__, we can now use **

# derivative with respect to x
x, a, b = Variable(24, 1), Variable(22, 0), Variable(2, 0)
fx = f(x, a, b)

# derivative with respect to a
x, a, b = Variable(24, 0), Variable(22, 1), Variable(2, 0)
fa = f(x, a, b)

# derivative with respect to b
x, a, b = Variable(24, 0), Variable(22, 0), Variable(2, 1)
fb = f(x, a, b)

print(f"x: {fx}")
print(f"a: {fa}")
print(f"b: {fb}")


The values of the partial derivatives match exactly! 

However, it becomes pretty evident that this is very inefficient if we want to calculate the derivative of a function with respect to multiple variables (which is exactly what we need for neural nets!). To overcome this limitation, we can implement reverse-mode autodiff, that requires two passes through the variables, but it's most efficient in the case of many inputs and few outputs (just what we need!).

# Computational graph

In our implementation of the forward-mode, we did not pay a lot of attention to the computational graph that is built to calculate the values. For reverse-mode, however, we need to pay close attention to it because if we want to traverse the graph in reverse order, we need to know which operator is computed when.

Imagine we have 3 functions that depend on one another: $y = \color{orange}{f(}\color{green}{g(}\color{red}{h(}\color{blue}{x}\color{red}{)}\color{green}{)}\color{orange}{)}$

We can break this function down into its individual components:

$$
\begin{align}
w_0 &= x \\
w_1 &= \color{red}{h(}w_0\color{red}{)} \\
w_2 &= \color{green}{g(}w_1\color{green}{)} \\
w_3 &= \color{orange}{f(}w_2\color{orange}{)} = y
\end{align}
$$

and differentiate using the chain rule:

$$
\begin{align}
\frac{\partial y}{\partial x} &= \frac{\partial w_3}{\partial w_0} \\
&= \frac{\partial w_3}{\partial w_2}\frac{\partial w_2}{\partial w_1}\frac{\partial w_1}{\partial w_0} \\
&= \frac{\partial \color{orange}{f(}w_2\color{orange}{)}}{\partial w_2}
   \frac{\partial \color{green}{g(}w_1\color{green}{)}}{\partial w_1} 
   \frac{\partial \color{red}{h(}w_0\color{red}{)}}{\partial w_1}
\end{align}
$$

We can generalize this, and realize that we are computing the following recursive expression:

$$
\frac{\partial w_i}{\partial w_0} = \frac{\partial w_i}{\partial w_{i-1}}\frac{\partial w_{i-1}}{\partial w_0}
$$

If we start at $i=3$, we end up with the following expression:

$$
\begin{align}
\frac{\partial y}{\partial x} &= \left(\left(\frac{\partial y}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
&= \left(\left(\left(\frac{\partial y}{\partial w_1}\right)\frac{\partial w_1}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
&= \left(\left(\left(\left(\frac{\partial y}{\partial w_2}\right)\frac{\partial w_2}{\partial w_1}\right)\frac{\partial w_1}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
&= \left(\left(\left(\left(\left(\frac{\partial y}{\partial w_3}\right)\frac{\partial w_3}{\partial w_2}\right)\frac{\partial w_2}{\partial w_1}\right)\frac{\partial w_1}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
\end{align}
$$


## Gradient tape

Breaking down a function into elementary operations is creating an _evaluation trace_ (also known as a "tape"). Let's look at the trace for the function

$$y = f(x, a, b) = \frac{xa}{(x^2 + b)}$$

when we differentiate with respect to $x$ and evaluate at $(x, a, b) = (24, 22, 2)$.

We first break down the function into elementary oprations, called _primals_, and simultaneously evaluate the derivative of the primals, called _tangents_. We'll associate these results with the intermediary variables $v_i$ and $\dot{v_i}$, where

$$\dot{v_i} = \frac{\partial v_i}{\partial x}$$

Remember that in the forward-mode, we set the tangent of the target variable to 1 and the others to 0. These initial assigments are called the _seeds_. So for the derivative with respect to $x$, we set $\dot{x} = 1$, $\dot{a} = 0$, and $\dot{b} = 0$.

![](images/autodiff-tutorial/table.png)
<!-- This renders horribly in jupyter, but here is the code to generate this table
| Variable | Elementary operation | Primals         | Variable    | Elementary operation                           | Tangents |
| -------- | -------------------- | -------         | --------    | --------------------                           | -------- |
| $v_0$    | $=x$                 | $=24$           | $\dot{v_0}$ | $=\dot{x}$                                     | $=1$ |
| $v_1$    | $=a$                 | $=22$           | $\dot{v_1}$ | $=\dot{a}$                                     | $=0$ |
| $v_2$    | $=b$                 | $=2$            | $\dot{v_2}$ | $=\dot{b}$                                     | $=0$ |
| $v_3$    | $=v_0 \times v_1$    | $=24 \times 22$ | $\dot{v_3}$ | $=\dot{v_0} \times v_1 +  v_0 \times \dot{v_1}$ | $=1 \times 22 + 24 \times 0$ |
| $v_4$    | $=v_0^2$             | $=24^2$         | $\dot{v_4}$ | $=2 \times v_0 \times \dot{v_0}$                | $=2 \times 24 \times 1$ |
| $v_5$    | $=v_2 + v_4$         | $=2 + 576$      | $\dot{v_5}$ | $=\dot{v_2} + \dot{v_4}$                        | $=0 + 48$ |
| $v_6$    | $=v_3 / v_5$         | $=528 / 578$    | $\dot{v_6}$ | $=(\dot{v_3} \times v_5 - v_3 \times \dot{v_5})/v_5^2$ | $=(22 \times 578 - 528 \times 48)/578^2$ |
| $y$      | $=v_6$               | $=0.9134$       | $\dot{y}$   | $=\dot{v_6}$ | $=-0.037$ | 
-->

We can visualize this evaluation trace by representing it as a computational graph (a directed acyclic graph):

![](images/autodiff-tutorial/computation-graph.png)

It should be pretty clear that constructing the graph explicitly is not required for forward-mode autodiff, but it is crucial for reverse-mode. For now, let's add some functionality to `Variable` so that we can trace the computation.

First, we simply add a class member to store the `parents` of the variable in the graph, the `name` of the variable, and the operation `op` that we perform.

In [ ]:
var_id = 0 # global variable that we can use to name variables automatically

class Variable():
    def __init__(self, f, d = 0, children=(), op = ''):
        global var_id
        self.f = f # evaluation
        self.d = d # tangent
        self.op = op # operation
        self.parents = set(children)
        self.name = f"v{var_id}"
        var_id += 1

    def __repr__(self):
        return f"Var(f={self.f}, d={self.d}, name={self.name}, op={self.op}, n_parents={len(self.parents)})"

And we can add all the operators we want:

In [ ]:
# binary
Variable.__add__ = lambda self, other: Variable(self.f + other.f, self.d + other.d, (self, other), "+")
Variable.__sub__ = lambda self, other: Variable(self.f - other.f, self.d - other.d, (self, other), "-")
Variable.__mul__ = lambda self, other: Variable(self.f * other.f, self.d * other.f + self.f * other.d, (self, other), "*")
Variable.__truediv__ = lambda self, other: Variable(self.f / other.f, (self.d * other.f - self.f * other.d) / other.f ** 2, (self, other), "/")

# unary
Variable.sin = lambda self: Variable(np.sin(self.f), np.cos(self.f) + self.d, (self,), "sin")
Variable.cos = lambda self: Variable(np.cos(self.f), -np.sin(self.f) + self.d, (self,), "cos")
Variable.log = lambda self: Variable(np.log(self.f), 1 / self.f * self.d, (self,), "log")
Variable.exp = lambda self: Variable(np.exp(self.f), np.exp(self.f) * self.d, (self,), "exp")

def var_pow(self, number):
    assert isinstance(number, (int, float))
    return Variable(np.power(self.f, number), number * np.power(self.f, number -1) * self.d, (self,), f"^{number}")

Variable.__pow__ = var_pow
Variable.__neg__ = lambda self: self * -1


We've added a bunch of stuff to the implementation, but so far it still implements the forward-mode:

In [ ]:
def f(x, a, b):
    return x * a / (x ** 2 + b) # because we implemented __pow__, we can now use **

# derivative with respect to x
x, a, b = Variable(24, 1), Variable(22), Variable(2)
fx = f(x, a, b)
print(f"fx : {fx}")

While this does offer a hint into the structure (we at least know that there are 2 parents for `fx`, we still don't see the full trace to know exactly all the steps that we took to make the calculation. For example, we can't compute $v_6$ before $v_3$ and $v_5$. $v_5$, in turn, cannot be computed before $v_4$. As we've seen in a previous lab, this can be done using the topological sort:

In [ ]:
def graph(self):
    topo_sorted = []
    visited = set()
    
    def topo_sort(v):
        if v not in visited:
            visited.add(v)
            for parent in v.parents:
                topo_sort(parent)
            topo_sorted.append(v)
    
    topo_sort(self)
    return topo_sorted
Variable.graph = graph

In [ ]:
fx.graph()

Although it should be pretty clear that the graph is stored in memor using the links we built using the variables, we are missing a way to visualize the actual graph. We will use `graphviz` to do this:

In [ ]:
from graphviz import Digraph

To visualize the graph, we need the nodes as well as the edges. We can collect these with a slightly modified version of our `graph` function:

In [ ]:
def trace(self):
    nodes, edges = set(), set()
    def traverse(v):
        if v not in nodes:
            nodes.add(v)
            for parent in v.parents:
                edges.add((parent, v))
                traverse(parent)
    traverse(self)
    return nodes, edges

Variable.trace = trace

In [ ]:
def draw(root, orientation="LR"):
    assert orientation in ["LR", "TB"] # left to right or top to bottom
    nodes, edges = root.trace()
    
    dot = Digraph(graph_attr={"rankdir":orientation})
    for n in nodes:
        dot.node(name=str(id(n)), label = f"{n.name}| f {n.f:.3f}| d {n.d:.3f}", shape="record")
        if n.op:
            dot.node(name=str(id(n)) + n.op, label=n.op)
            dot.edge(str(id(n)) + n.op, str(id(n)))
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2.op)
        
    return dot

In [ ]:
draw(fx)

It looks remarkably like the graph we saw earlier. But we are not calculating the gradients in reverse-mode yet. Let's do that now:

# Reverse-mode

So far, we've implemented the forward-mode and a way to trace the computational graph that computes all the primals and tangents.
Now, we simply need to compute the gradients by starting from the output of the evaluation trace, and traverse towards the inputs.
Luckily, adding this to our implementation is not difficult, as we need to:
1. Compute the function evaluation to start computing the gradient in the reverse direction
2. The primal trace graph to traverse backwards
3. Implement the operation we have to perform to compute the gradient.

1 and 2 are implemented in the forward-mode. And with minimal modifications. First, we no longer need the forward accumulation of the gradients, and therefore we don't need to compute this. Then, we need to remember which operations we need to perform in the reverse direction. We simply store the function as a member of the node to run when we do the backward pass. And that's it!

First, let's look at the differences between forward-mode and reverse-mode. In reverse-mode, each intermediate variable $v_i$ is associated with an _adjoint_ $\bar{v}_i$ (note that this is different from the _tangent_ in forward mode:

$$\bar{v}_i = \frac{\partial y_j}{\partial v_i}$$

The adjoint represents the sensitivity of an output $y_j$ with respect to changes in $v_i$. Again, imagine we have 3 functions that depend on one another: $y = \color{orange}{f(}\color{green}{g(}\color{red}{h(}\color{blue}{x}\color{red}{)}\color{green}{)}\color{orange}{)}$

We can break this function down into its individual components:

$$
\begin{align}
w_0 &= x \\
w_1 &= \color{red}{h(}w_0\color{red}{)} \\
w_2 &= \color{green}{g(}w_1\color{green}{)} \\
w_3 &= \color{orange}{f(}w_2\color{orange}{)} = y
\end{align}
$$

and differentiate using the chain rule:

$$
\begin{align}
\frac{\partial y}{\partial x} &= \frac{\partial w_3}{\partial w_0} \\
&= \frac{\partial w_3}{\partial w_2}\frac{\partial w_2}{\partial w_1}\frac{\partial w_1}{\partial w_0} \\
&= \frac{\partial \color{orange}{f(}w_2\color{orange}{)}}{\partial w_2}
   \frac{\partial \color{green}{g(}w_1\color{green}{)}}{\partial w_1} 
   \frac{\partial \color{red}{h(}w_0\color{red}{)}}{\partial w_1}
\end{align}
$$

We can generalize this, and realize that we are computing the following recursive expression:

$$
\frac{\partial w_3}{\partial w_i} = \frac{\partial w_3}{\partial w_{i+1}}\frac{\partial w_{i+1}}{\partial w_i}
$$

Again, starting from $i=0$, we achieve the nasty expression:

$$
\begin{align}
\frac{\partial y}{\partial x} &= \left(\left(\frac{\partial y}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
&= \left(\left(\left(\frac{\partial y}{\partial w_1}\right)\frac{\partial w_1}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
&= \left(\left(\left(\left(\frac{\partial y}{\partial w_2}\right)\frac{\partial w_2}{\partial w_1}\right)\frac{\partial w_1}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
&= \left(\left(\left(\left(\left(\frac{\partial y}{\partial w_3}\right)\frac{\partial w_3}{\partial w_2}\right)\frac{\partial w_2}{\partial w_1}\right)\frac{\partial w_1}{\partial w_0}\right)\frac{\partial w_0}{\partial x}\right) \\
\end{align}
$$

Let's compute the derivatives in reverse-mode for our function:

$$y = f(x, a, b) = \frac{xa}{(x^2 + b)}$$

![](images/autodiff-tutorial/forward.png)

![](images/autodiff-tutorial/backward.png)

Note that when we ran the computations in reverse-mode, we ended up with the partial derivatives for all inputs! 

A case such as this function, where the output is a scalar is a special case of automatic differentiation that we know as backpropagation!

Now, let's code this into our differentiation engine. For each node (operation), we will define a `backward` opration using the current accumulated gradient `d` of `self` and `other`

In [ ]:
var_id = 0

class Variable():
    def __init__(self, f, children = (), op = ''):
        global var_id
        self.f = f # evaluation
        self.d = 0 # adjoint
        self.op = op 
        self.backward_fn = lambda: None # backwards function
        self.parents = set(children) 
        self.parents = set(children)
        self.name = f"v{var_id}"
        var_id += 1
    
    def __repr__(self):
        return f"Var(f={self.f}, d={self.d}, name={self.name}, op={self.op}, n_parents={len(self.parents)}), is_leaf={'False' if len(self.parents) else 'True'}"
        
    
    # I will add only the operators we need for the function we're studying, but feel free to add more operators (reuse the ones implemented in the forward-mode)
    
    def __add__(self, other):
        other = other if isinstance(other, Variable) else Variable(other)
        out = Variable(self.f + other.f, (self, other), '+')

        def _backward_fn():
            self.d  += out.d
            other.d += out.d
        out.backward_fn = _backward_fn
        
        return out
    
    def __sub__(self, other):
        other = other if isinstance(other, Variable) else Variable(other)
        out = Variable(self.f - other.f, (self, other), '-')

        def _backward_fn():
            self.d  += out.d
            other.d -= out.d
        out.backward_fn = _backward_fn

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Variable) else Variable(other)
        out = Variable(self.f * other.f, (self, other), '*')

        def _backward_fn():
            self.d  += other.f * out.d
            other.d += self.f * out.d
        out.backward_fn = _backward_fn

        return out
    def __truediv__(self, other):
        out = Variable(self.f / other.f, (self, other), "/")
        
        def _backward_fn():
            self.d  += 1 / other.f * out.d
            other.d += -self.f / other.f ** 2 * out.d

        out.backward_fn = _backward_fn

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Variable(self.f**other, (self,), f'^{other}')

        def _backward_fn():
            self.d += (other * self.f**(other-1)) * out.d
        out.backward_fn = _backward_fn

        return out
    
    
    def backward(self):
        tape = self.graph()
        
        self.d = 1
        
        for v in reversed(tape):
            print(v)
            v.backward_fn()

# We redefined this class, let's reuse the implementation from before            
Variable.graph = graph
Variable.trace = trace


Now, let's compute the forward pass as usual (the gradients are not populated yet)

In [ ]:
def f(x, a, b):
    return x * a / (x ** 2 + b) # because we implemented __pow__, we can now use **

x, a, b = Variable(24), Variable(22), Variable(2)
y = f(x, a, b)
print(f"y : {y}")

In [ ]:
y.backward()

To finish, let's draw the graph in reverse mode, so we can observe what's going on without having to read the list in a non-intuitive order, this function is the same, only we reverse the direction of the edges

In [ ]:
def draw(root, orientation="RL"):
    assert orientation in ["LR", "TB", "RL"] # left to right, top to bottom, or right to left
    nodes, edges = root.trace()
    
    dot = Digraph(graph_attr={"rankdir":orientation})
    for n in nodes:
        dot.node(name=str(id(n)), label = f"{n.name}| f {n.f:.3f}| d {n.d:.3f}", shape="record")
        if n.op:
            dot.node(name=str(id(n)) + n.op, label=n.op)
            dot.edge(str(id(n)), str(id(n)) + n.op)
    for n1, n2 in edges:
        dot.edge(str(id(n2)) + n2.op, str(id(n1)))
        
    return dot

In [ ]:
draw(y)

# Next steps (optional exercises for home, highly recommended)

You now have a differentiation engine to play around with. 

Try to implement

* a Linear Regression
* a Single Perceptron
* a MultiLayer Perceptron

By composing these functions, you should be able to do this. I highly encourage you to try and think about it carefully before looking for references, it's a great exercise!

If you're totally lost, have a read at the [micrograd demo](https://github.com/karpathy/micrograd/blob/master/demo.ipynb). That one uses a smaller engine, but with a very similar API.